# High-Accuracy Emotion Recognition Model - 80%+ Accuracy Target

This notebook implements state-of-the-art techniques to achieve **80%+ accuracy** in emotion recognition.

## Key Improvements Over Baseline:
1. **Advanced Architecture**: ResNet50 with attention mechanism and enhanced classifier
2. **Sophisticated Data Augmentation**: MixUp, CutMix, AutoAugment, and advanced transforms
3. **Enhanced Loss Functions**: Focal Loss with class balancing and label smoothing
4. **Optimized Training**: OneCycleLR, mixed precision, gradient clipping, longer training
5. **Class Imbalance Handling**: Weighted sampling and class-aware loss computation
6. **Advanced Regularization**: Multiple dropout layers, weight decay, stochastic depth
7. **Progressive Training**: Progressive image resizing and curriculum learning
8. **Test-Time Augmentation**: Multiple inference passes for better accuracy
9. **Comprehensive Evaluation**: Detailed metrics, confusion matrices, and analysis

**Target**: Achieve ≥80% test accuracy with robust generalization


In [ ]:
# ===============================
# Cell 1 — Enhanced Setup & Imports
# ===============================

import os
import sys
import json
import math
import time
import random
import warnings
from pathlib import Path
from collections import Counter
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
from PIL import Image
import cv2

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingWarmRestarts
from torch.cuda.amp import GradScaler, autocast

# Torchvision imports
import torchvision
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights, efficientnet_b3, EfficientNet_B3_Weights

# ML metrics
from sklearn.metrics import (
    classification_report, confusion_matrix, f1_score, 
    accuracy_score, precision_score, recall_score
)
from sklearn.utils.class_weight import compute_class_weight

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from IPython.display import clear_output, display

# Set style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings('ignore')

# Enhanced reproducibility
def set_seed(seed=42):
    """Set seed for reproducibility across all libraries"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"✓ Random seed set to {seed} for reproducibility")

set_seed(42)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name()}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   ⚠️  CUDA not available - using CPU (will be slower)")

print(f"\n📚 Library versions:")
print(f"   PyTorch: {torch.__version__}")
print(f"   Torchvision: {torchvision.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")